# NTD 2021 vs 2022

* Explore where / how much `ntd_id` has changed between 2021 and 2022 exports. 
* Use BigQuery, from `mart_ntd` grab 2021 and 2022 and export as csv.
* Pass it through a variety of merges can help winnow down which ones we do need to manually reconcile.
* Go from more stringent merges (ids and names) to looser merges
   * Parsing the `ntd_id` and grabbing the suffix portion can help, since there's a good batch where the `ntd_id` change in 2022 is a new prefix added, where `ntd_id_2022 = [xxxx-ntd_id_2021]`.

In [1]:
import numpy as np
import pandas as pd

#GCS_BUCKET = "gs://calitp-analytics-data/data-analyses/ntd"
GCS_PATH = "gs://calitp-analytics-data/data-analyses/ntd/"
FILE_NAMEa = "ntd_2021_2022.csv"
FILE_NAMEb = "manual.csv"

LOCAL_PATH = "ntd_2021_2022.csv"

In [2]:
df_full = pd.read_csv(
    f"{GCS_PATH}{FILE_NAMEa}")

df_2021 = df_full[df_full.year==2021].reset_index(drop=True)
df_2022 = df_full[df_full.year==2022].reset_index(drop=True)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
def basic_stats(df: pd.DataFrame): 
    cols = ["ntd_id", "legacy_ntd_id", 
            "reported_by_name", 
            "agency_name",
            "city"
           ]
    for c in cols:
        print(f"nunique {c}: {df[c].nunique()}")

In [4]:
basic_stats(df_2021)

nunique ntd_id: 3021
nunique legacy_ntd_id: 2110
nunique reported_by_name: 64
nunique agency_name: 2929
nunique city: 1974


In [5]:
basic_stats(df_2022)

nunique ntd_id: 2969
nunique legacy_ntd_id: 2092
nunique reported_by_name: 60
nunique agency_name: 2924
nunique city: 1964


## Full set of merge columns

* `ntd_id, legacy_ntd_id, agency_name, reported_by_name, city`

Probably the most complete set of identifiers

In [6]:
cols = ["ntd_id", "legacy_ntd_id", 
        "reported_by_name", 
        "agency_name",
        "city", 
       ]

m1 = pd.merge(
    df_2021[cols + ["key", "year"]],
    df_2022[cols + ["key", "year"]],
    on = cols,
    how = "outer",
    indicator = True
)

m1._merge.value_counts()

left_only     1891
right_only    1887
both          1130
Name: _merge, dtype: int64

In [7]:
m1._merge.value_counts(normalize=True)

left_only     0.385289
right_only    0.384474
both          0.230236
Name: _merge, dtype: float64

### Majority will merge if we don't merge on `ntd_id`, but use `legacy_ntd_id` and variations of name instead

These could be solved if we just use `agency_name` and `reported_by_name`. Even though `ntd_id` is not necessarily the same, `legacy_ntd_id` appears to be (even if same means it's NaN for both years).

Exclude `city` from merge, since there are some that change cities, but it's the same agency. We do want to know if city changes from year to year.

In [8]:
m1[m1._merge != "both"].sort_values("agency_name")

,ntd_id,legacy_ntd_id,reported_by_name,agency_name,city,key_x,year_x,key_y,year_y,_merge
3614,70242,7R01-015,Iowa Department of Transportation,10-15 Regional Transit Agency,Ottumwa,NaN,NaN,49ba215d3793e5a58366b7dde1cb587e,2022.0,right_only
2002,7R01-70242,7R01-015,Iowa Department of Transportation,10-15 Regional Transit Agency,Ottumwa,4400da6a95a6a328ab9b7f651a1bdeb4,2021.0,NaN,NaN,left_only
963,A0002-55329,NaN,Stark Area Regional Transit Authority,"ABCD, Inc.",Canton,463265c2d666621a7757630e5548fc3b,2021.0,NaN,NaN,left_only
4536,55329,NaN,Stark Area Regional Transit Authority,"ABCD, Inc.",Canton,NaN,NaN,16b0e8ca1108b744db0f7952d8f6b381,2022.0,right_only
3211,88285,NaN,Colorado Department of Transportation,AEX - Alpine Express,Gunnison,NaN,NaN,00fc807db5fbd510c447e71e21069b19,2022.0,right_only
...,...,...,...,...,...,...,...,...,...,...
2182,9R02-91070,9R02-019,California Department of Transportation,Yosemite Area Regional Transportation System,Merced,f68cf9996e537ff2968339f4508dd60d,2021.0,NaN,NaN,left_only
4269,91070,9R02-019,California Department of Transportation,Yosemite Area Regional Transportation System,Merced,NaN,NaN,1e8e914f03d459189a27bcdc2fc32106,2022.0,right_only
4422,66320,NaN,Texas Department of Transportation,Zapata County,Zapata,NaN,NaN,e19ba955b67fff1dc3fd906aad5ab873,2022.0,right_only
1390,6R05-66320,NaN,Texas Department of Transportation,Zapata County,Zapata,dc5f81697aabb1d127a4c03225a6cdde,2021.0,NaN,NaN,left_only


In [9]:
# Remove the keys that would now merge between the years
ok_keys = np.concatenate((
    m1[m1._merge=="both"].key_x.unique(),
    m1[m1._merge=="both"].key_y.unique()
))

### Merge on `legacy_ntd_id`, variations of name
These probably need to be manually addressed using a crosswalk, since we want to store variations of the `agency_name` over time.

In [10]:
# Remove city from merge, since there are a couple
# that would merge but are set to diff cities
m2 = pd.merge(
    df_2021[~df_2021.key.isin(ok_keys)][cols + ["key", "year"]],
    df_2022[~df_2022.key.isin(ok_keys)][cols + ["key", "year"]],
    on = ["legacy_ntd_id", "reported_by_name", "agency_name"],
    how = "outer",
    indicator = True
)

m2._merge.value_counts()

both          1789
left_only      102
right_only      98
Name: _merge, dtype: int64

Some of these are clearly the same agency when you spot check it (abbreviations, minor changes in name, etc), but some are less obvious. Might have to start compiling a larger crosswalk of variations on agency name.

These would be grouped together (`ntd_id` changes...but with an additional prefix)
* Whitley County Commissioners
* Whitley County Council on Aging

In [11]:
m2[m2._merge != "both"].sort_values(["agency_name"])

,ntd_id_x,legacy_ntd_id,reported_by_name,agency_name,city_x,key_x,year_x,ntd_id_y,city_y,key_y,year_y,_merge
1342,6R02-66284,NaN,Louisiana Department of Transportation,Acadia COA,Crowley,5127e6c6b2c3c3018cc2629cfa94f74e,2021.0,NaN,NaN,NaN,NaN,left_only
1692,A0004-00415,NaN,Valley Regional Transit,Ada County Highway District,Boise,8b1ef6431abe5e9a5eb5c0a1124477d9,2021.0,NaN,NaN,NaN,NaN,left_only
1928,NaN,5R02-020,Indiana Department of Transportation,Area 10 Council on Aging of Monroe County,NaN,NaN,NaN,50308,Ellettsville,00d4f9e499a70f66a8a72c0f4c42d408,2022.0,right_only
1971,NaN,5R02-017,Indiana Department of Transportation,Area IV Agency on Aging and Community Action P...,NaN,NaN,NaN,50365,Lafayette,a561a8b8df6a37f2dd4f141e4902d346,2022.0,right_only
1937,NaN,NaN,Arizona Department of Transportation,Assist to Independence,NaN,NaN,NaN,99466,Tuba City,42b9d82b9be91d4b150f55f9e88a809f,2022.0,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...
1577,5R02-50468,5R02-039,Indiana Department of Transportation,Whitley County Commissioners,Columbia City,518ace07cb87072ec541dc7fef24e6f7,2021.0,NaN,NaN,NaN,NaN,left_only
1973,NaN,5R02-039,Indiana Department of Transportation,Whitley County Council on Aging,NaN,NaN,NaN,50468,Columbia City,0b058ae9f5d5a1e9780257c981a6b91c,2022.0,right_only
1686,6R02-66299,NaN,Louisiana Department of Transportation,Winn COA,Winnfield,57e123ed53fdf3ce74769f216cc2c39d,2021.0,NaN,NaN,NaN,NaN,left_only
1933,NaN,5R02-024,Indiana Department of Transportation,YMCA of Vincennes,NaN,NaN,NaN,50392,Vincennes,13d170d217ad0cf7860e62ce13651004,2022.0,right_only


In [12]:
m2[m2._merge != "both"].sort_values(["agency_name"]).agency_name.unique()

array(['Acadia COA', 'Ada County Highway District',
       'Area 10 Council on Aging of Monroe County',
       'Area IV Agency on Aging and Community Action Programs ',
       'Assist to Independence', 'Atlanta-Region Transit Link Authority',
       'Autonomous Municipality of Vega Alta', 'Bacon County',
       'Baltimore County Department of Aging',
       'Baltimore County Department of Public Works Transportation',
       'Bay State Cruise Company', 'Bay State LLC', 'Berrien County',
       'Blue River Services ', 'Boone County Commissioners',
       'Boone County Senior Services', 'Boulder, City of',
       'Brantley County', 'Brooks County Transit',
       'Brown County Senior Citizens Council', 'Brown County YMCA',
       'Buckeye Community Services', 'CENTRAL MISSISSIPPI  INC',
       'CHANDLER, CITY OF', 'Cache Employment & Training Center (CETC)',
       'Calcasieu Voluntary Council in Aging', 'Cardinal Services ',
       'Cass County Commissioners', 'Cass County Council on Ag

In [13]:
# Remove the keys that would now merge between the years
ok_keys2 = np.concatenate((
    ok_keys,
    m2[m2._merge == "both"].key_x.unique(),
    m2[m2._merge == "both"].key_y.unique()
))

In [14]:
def ntd_id_parsed(df: pd.DataFrame):
    df = df.assign(
        ntd_id_no_prefix = df.apply(
            lambda x:
            x.ntd_id.split("-")[1] if "-" in x.ntd_id
            else x.ntd_id, 
            axis=1)
    )
    
    return df

In [15]:
m3 = pd.merge(
    df_2021[~df_2021.key.isin(ok_keys2)][cols + ["key", "year"]].pipe(ntd_id_parsed),
    df_2022[~df_2022.key.isin(ok_keys2)][cols + ["key", "year"]].pipe(ntd_id_parsed),
    on = ["ntd_id_no_prefix", "legacy_ntd_id",],
    how = "outer",
    indicator = True
)

m3._merge.value_counts()

left_only     61
right_only    54
both          41
Name: _merge, dtype: int64

In [16]:
m3.head(2)

,ntd_id_x,legacy_ntd_id,reported_by_name_x,agency_name_x,city_x,key_x,year_x,ntd_id_no_prefix,ntd_id_y,reported_by_name_y,agency_name_y,city_y,key_y,year_y,_merge
0,5R02-50399,5R02-004,Indiana Department of Transportation,Fayette Community Commissioners,Connersville,eeae713c46c405e3c8d4f001df04ab08,2021.0,50399,50399,Indiana Department of Transportation,Fayette County Council Agency on Aging and Aged,Connersville,4e3cd53439c5b16a8db255353ab5f452,2022.0,both
1,167,0T19,NaN,Native Village of Crooked Creek,Crooked Creek,5f174c276fcb5bdafe25d2d81bc90dc9,2021.0,167,NaN,NaN,NaN,NaN,NaN,NaN,left_only


### Parsing `ntd_id` into a no-prefix version can help 

If we are going to remove the prefix...we could do this earlier and hopefully get more to merge. Although, this does mean that we are left with variations on `agency_name` and `reported_by_name`, which still needs to make it into our crosswalk, even if we do not use it to merge.

Left with a batch of about 60 to reconcile manually.

In [17]:
m1_table = m1.loc[m1['_merge'] == "both", ['ntd_id', 'reported_by_name','legacy_ntd_id','agency_name','city']]

In [18]:
m1_table

,ntd_id,reported_by_name,legacy_ntd_id,agency_name,city
1,11239,NaN,NaN,"Hyannis Harbor Tours, Inc.",Hyannis
2,21,NaN,21,Whatcom Transportation Authority,Bellingham
6,30010,NaN,3010,Lehigh and Northampton Transportation Authority,Allentown
7,30041,NaN,3041,Board of Commissioners of Allegany County Mary...,Cumberland
8,30201,NaN,NaN,City of Baltimore,Baltimore
...,...,...,...,...,...
2999,50020,NaN,5020,"City of Springfield, Ohio",Springfield
3000,50047,NaN,5047,Bloomington-Normal Public Transit System,Normal
3007,66194,NaN,6T14,Cheyenne & Arapaho Tribes,Concho
3009,70032,NaN,7032,"City of St. Joseph, Missouri",Saint Joseph


### Merging all the crosswalks to create one table

In [19]:
m1_table['ntd_id_2021'] = m1_table['ntd_id']

In [20]:
m1_table.rename(columns={'ntd_id' : 'ntd_id_2022'}, inplace = True)

In [21]:
m1_table

,ntd_id_2022,reported_by_name,legacy_ntd_id,agency_name,city,ntd_id_2021
1,11239,NaN,NaN,"Hyannis Harbor Tours, Inc.",Hyannis,11239
2,21,NaN,21,Whatcom Transportation Authority,Bellingham,21
6,30010,NaN,3010,Lehigh and Northampton Transportation Authority,Allentown,30010
7,30041,NaN,3041,Board of Commissioners of Allegany County Mary...,Cumberland,30041
8,30201,NaN,NaN,City of Baltimore,Baltimore,30201
...,...,...,...,...,...,...
2999,50020,NaN,5020,"City of Springfield, Ohio",Springfield,50020
3000,50047,NaN,5047,Bloomington-Normal Public Transit System,Normal,50047
3007,66194,NaN,6T14,Cheyenne & Arapaho Tribes,Concho,66194
3009,70032,NaN,7032,"City of St. Joseph, Missouri",Saint Joseph,70032


Filtering rows with successful crosswalk and selecting few columns to concatenate

In [22]:
m2_table = m2.loc[m2['_merge'] == "both", ['ntd_id_x', 'reported_by_name','legacy_ntd_id','agency_name','city_y','ntd_id_y']]

In [23]:
m2_table.rename(columns={'ntd_id_y' : 'ntd_id_2022', 'ntd_id_x' : 'ntd_id_2021', 'city_y' : 'city'}, inplace = True)

In [24]:
col_order = ['ntd_id_2022', 'reported_by_name','legacy_ntd_id','agency_name','city', 'ntd_id_2021']
m2_table = m2_table[col_order]

In [25]:
m3_table = m3.loc[m3['_merge'] == "both", ['ntd_id_x', 'reported_by_name_y','legacy_ntd_id','agency_name_y','city_y','ntd_id_y']]

In [26]:
m3_table.rename(columns={'ntd_id_y' : 'ntd_id_2022', 'ntd_id_x' : 'ntd_id_2021', 'city_y' : 'city', 'reported_by_name_y' : 'reported_by_name', 'agency_name_y' : 'agency_name' }, inplace = True)

In [27]:
m3_table = m3_table[col_order]

Creating the concatenated table 

In [28]:
crosswalk = pd.concat([m1_table,m2_table,m3_table], axis=0)

In [29]:
crosswalk

,ntd_id_2022,reported_by_name,legacy_ntd_id,agency_name,city,ntd_id_2021
1,11239,NaN,NaN,"Hyannis Harbor Tours, Inc.",Hyannis,11239
2,21,NaN,21,Whatcom Transportation Authority,Bellingham,21
6,30010,NaN,3010,Lehigh and Northampton Transportation Authority,Allentown,30010
7,30041,NaN,3041,Board of Commissioners of Allegany County Mary...,Cumberland,30041
8,30201,NaN,NaN,City of Baltimore,Baltimore,30201
...,...,...,...,...,...,...
84,50468,Indiana Department of Transportation,5R02-039,Whitley County Council on Aging,Columbia City,5R02-50468
87,50483,Indiana Department of Transportation,5R02-005,Franklin County Senior Services,Brookville,5R02-50483
94,50308,Indiana Department of Transportation,5R02-020,Area 10 Council on Aging of Monroe County,Ellettsville,5R02-50308
96,50276,Indiana Department of Transportation,5R02-032,Four Rivers Resource Services,Loogootee,5R02-50276


#### Checking for duplicated rows

In [30]:
duplicated_rows = crosswalk.duplicated()

In [31]:
print("duplicated_rows:")
print(crosswalk[duplicated_rows])

duplicated_rows:
Empty DataFrame
Columns: [ntd_id_2022, reported_by_name, legacy_ntd_id, agency_name, city, ntd_id_2021]
Index: []


In [32]:
duplicated_rows_subset = crosswalk.duplicated(subset = ['ntd_id_2022', 'agency_name', 'reported_by_name'])

In [33]:
print("duplicated_rows_subset:")
print(crosswalk[duplicated_rows_subset])

duplicated_rows_subset:
Empty DataFrame
Columns: [ntd_id_2022, reported_by_name, legacy_ntd_id, agency_name, city, ntd_id_2021]
Index: []


#### Manual crosswalk for the remaining ntd_ids


- Pioneer valley transit authority intercity bus reported by Massachusetts Department of Transportation has incorrect ntd_id (checked from the NTD profile data)
- Some agencies do not have 2021 data but has 2020 data. 


In [34]:
m4_table = pd.read_csv(
    f"{GCS_PATH}{FILE_NAMEb}")

In [35]:
m4_table.head(2)

,ntd_id_2022,reported_by_name,legacy_ntd_id,agency_name,city,ntd_id_2021,Remarks
0,231,NaN,0T33,Confederated Tribes of Warm Springs,Warm Springs,NaN,No 2021 data
1,90306,NaN,NaN,Stanislaus Regional Transit Authority,Modesto,NaN,the county doesn’t have data after 2021 so cou...


In [36]:
crosswalk['Remarks'] = pd.Series(dtype=float)

In [37]:
crosswalk.head(2)

,ntd_id_2022,reported_by_name,legacy_ntd_id,agency_name,city,ntd_id_2021,Remarks
1,11239,NaN,NaN,"Hyannis Harbor Tours, Inc.",Hyannis,11239,NaN
2,21,NaN,21,Whatcom Transportation Authority,Bellingham,21,NaN


### Final Concatenation

In [38]:
final_crosswalk = pd.concat([crosswalk, m4_table], axis=0)

In [39]:
final_crosswalk

,ntd_id_2022,reported_by_name,legacy_ntd_id,agency_name,city,ntd_id_2021,Remarks,ntd_id_2022,Remarks
1,11239,NaN,NaN,"Hyannis Harbor Tours, Inc.",Hyannis,11239,NaN,NaN,NaN
2,21,NaN,21,Whatcom Transportation Authority,Bellingham,21,NaN,NaN,NaN
6,30010,NaN,3010,Lehigh and Northampton Transportation Authority,Allentown,30010,NaN,NaN,NaN
7,30041,NaN,3041,Board of Commissioners of Allegany County Mary...,Cumberland,30041,NaN,NaN,NaN
8,30201,NaN,NaN,City of Baltimore,Baltimore,30201,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
47,NaN,Arizona Department of Transportation,NaN,New Horizons Disability Empowerment Center,Prescott Valley,NaN,NaN,99463.0,NaN
48,NaN,Utah Department of Transportation,NaN,Kane County Senior Citizens Improvement Corp,Kanab,NaN,NaN,88253.0,NaN
49,NaN,Arizona Department of Transportation,NaN,Saguaro Foundation Community Living Programs,Yuma,NaN,NaN,99462.0,NaN
50,NaN,Arizona Department of Transportation,NaN,Town of springerville AZ,Springerville,NaN,NaN,99467.0,NaN


In [40]:
final_crosswalk.to_csv(f"{GCS_PATH}final_crosswalk.csv")